<a href="https://colab.research.google.com/github/ShaunakSen/Deep-Learning/blob/master/seven_part_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Sequence Modelling

__Before starting, we recommend you enable GPU acceleration if you're running on Colab.__

In [1]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 92kB 9.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/6c/cb/69/466aef9cee879fb8f645bd602e34d45e754fb3dee2cb1a877a
Successfully built torchbearer


## Markov chains

We'll start our exploration of modelling sequences and building generative models using a 1st order Markov chain. The Markov chain is a stochastic model describing a sequence of possible events in which the probability of each event depends only on the state attained in the previous event. In our case we're going to learn a model over a set of characters from an English language text. The events, or states, in our model are the set of possible characters, and we'll learn the probability of moving from one character to the next.

Let's start by loading the data from the web:

In [2]:
from torchvision.datasets.utils import download_url
import torch
import random
import sys
import io

# Read the data
download_url('https://s3.amazonaws.com/text-datasets/nietzsche.txt', '.', 'nietzsche.txt', None)
text = io.open('./nietzsche.txt', encoding='utf-8').read().lower()
print('corpus length:', len(text))

0it [00:00, ?it/s]

606208it [00:00, 720752.62it/s]                            

corpus length: 600893


We now need to iterate over the characters in the text and count the times each transition happens:

In [0]:
transition_counts = dict()
for i in range(0,len(text)-1):
    currc = text[i]
    nextc = text[i+1]
    if currc not in transition_counts:
        transition_counts[currc] = dict()
    if nextc not in transition_counts[currc]:
        transition_counts[currc][nextc] = 0
    transition_counts[currc][nextc] += 1

In [4]:
print (transition_counts)

{'p': {'r': 1533, 'p': 421, 'o': 1259, 'e': 1901, 'h': 778, 'a': 822, '.': 10, 'i': 632, 'u': 314, 's': 321, 'l': 790, 't': 417, ',': 31, ' ': 157, 'y': 23, '\n': 13, 'n': 6, 'm': 30, '?': 1, 'w': 5, 'b': 1, 'f': 7, 'g': 1, '"': 2, ';': 2, '-': 4, ':': 3}, 'r': {'e': 7222, 'u': 562, 'o': 1987, ' ': 4027, 's': 1337, 'r': 325, 'i': 2450, 't': 1289, '\n': 362, 'y': 997, 'a': 2279, 'h': 210, 'm': 552, 'd': 797, ',': 501, 'w': 52, 'l': 337, 'v': 170, '-': 141, 'c': 274, 'p': 158, 'n': 434, '?': 24, 'f': 141, '.': 111, 'g': 130, 'k': 116, ')': 10, '!': 15, ':': 35, ';': 25, 'b': 83, '"': 25, "'": 33, '_': 6, '[': 2, ']': 3, 'x': 1, '=': 1}, 'e': {'f': 641, '\n': 1571, 'n': 5574, 'r': 7885, ' ': 15665, 'c': 1468, 'y': 555, 'e': 1334, 'd': 3223, 's': 5421, 'i': 857, 'm': 1311, 't': 1348, 'v': 1566, 'l': 2885, ',': 1417, 'a': 2590, 'g': 417, 'p': 569, '.': 374, '-': 270, 'u': 153, 'o': 231, '"': 89, 'x': 756, 'w': 342, 'j': 30, '?': 79, 'z': 5, ';': 92, '!': 69, 'h': 97, '_': 11, 'b': 120, 'q':

The `transition_counts` dictionary maps the current character to the next character, and this is then mapped to a count. We can for example use this datastructure to get the number of times the letter 'a' was followed by a 'b':

In [5]:
print("Number of transitions from 'a' to 'b': " + str(transition_counts['a']['b']))

Number of transitions from 'a' to 'b': 813


Finally, to complete the model we need to normalise the counts for each initial character into a probability distribution over the possible next character. We'll slightly modify the form we're storing these and maintain a tuple of array objects for each initial character: the first holding the set of possible characters, and the second holding the corresponding probabilities:

In [6]:
transition_probabilities = dict()
for currentc, next_counts in transition_counts.items():
    # next_counts is the dict of all transition chars
    values = []
    probabilities = []
    sumall = 0
    for nextc, count in next_counts.items():
        values.append(nextc)
        probabilities.append(count)
        sumall += count
    # normalize
    for i in range(0, len(probabilities)):
        probabilities[i] /= float(sumall)
    transition_probabilities[currentc] = (values, probabilities)
        
print(transition_probabilities)

{'p': (['r', 'p', 'o', 'e', 'h', 'a', '.', 'i', 'u', 's', 'l', 't', ',', ' ', 'y', '\n', 'n', 'm', '?', 'w', 'b', 'f', 'g', '"', ';', '-', ':'], [0.16164065795023197, 0.044390552509489666, 0.1327498945592577, 0.20044285111767188, 0.08203289751159848, 0.08667229017292281, 0.001054407423028258, 0.06663854913538592, 0.03310839308308731, 0.03384647827920709, 0.0832981864192324, 0.043968789540278365, 0.0032686630113876003, 0.016554196541543654, 0.002425137072964994, 0.0013707296499367355, 0.0006326444538169548, 0.0031632222690847742, 0.00010544074230282581, 0.000527203711514129, 0.00010544074230282581, 0.0007380851961197807, 0.00010544074230282581, 0.00021088148460565162, 0.00021088148460565162, 0.00042176296921130323, 0.0003163222269084774]), 'r': (['e', 'u', 'o', ' ', 's', 'r', 'i', 't', '\n', 'y', 'a', 'h', 'm', 'd', ',', 'w', 'l', 'v', '-', 'c', 'p', 'n', '?', 'f', '.', 'g', 'k', ')', '!', ':', ';', 'b', '"', "'", '_', '[', ']', 'x', '='], [0.26528063473405816, 0.020643549808992065, 0.0

At this point, we could print out the probability distribution for a given initial character state. For example, to print the distribution for 'a':

In [7]:
for a,b in zip(transition_probabilities['a'][0], transition_probabilities['a'][1]):
    print(a,b)

c 0.03685183172083922
t 0.14721708881400153
  0.05296771388194369
n 0.2322806826829003
l 0.11552886183280792
r 0.08794434177628004
s 0.0968583541689314
v 0.0192412218719426
i 0.03402543754755952
d 0.026986628981411024
g 0.017202956843135123
y 0.02505707142080661
k 0.012827481247961734
b 0.02209479291227307
p 0.020545711490379388
m 0.02030111968692249
u 0.011414284161321883
f 0.004429829329274921
w 0.004837482335036417
, 0.0010870746820306554

 0.005353842809000978
z 0.0006522448092183933
x 0.0007609522774214588
o 0.0005435373410153277
. 0.000489183606913795
- 0.0004348298728122622
' 5.4353734101532776e-05
j 0.0004348298728122622
h 0.00035329927165996303
e 0.0007337754103706925
: 5.4353734101532776e-05
a 5.4353734101532776e-05
) 0.00010870746820306555
! 2.7176867050766388e-05
; 2.7176867050766388e-05
" 8.153060115229916e-05
q 2.7176867050766388e-05
_ 8.153060115229916e-05
[ 2.7176867050766388e-05


In [8]:
# Verifying that  the most probable letter to follow an 'a' is 'n'
import numpy as np

transition_probabilities['a'][0][np.argmax(transition_probabilities['a'][1])]

'n'

It looks like the most probable letter to follow an 'a' is 'n'. 

__What is the most likely letter to follow the letter 'j'? Write your answer in the block below:__

The most likely letter to follow the letter 'j' is 'u'. The code is in the block below

In [9]:
transition_probabilities['j'][0][np.argmax(transition_probabilities['j'][1])]

'u'

We mentioned earlier that the Markov model is generative. This means that we can draw samples from the distributions and iteratively move between states. 

Use the following code block to iteratively sample 1000 characters from the model, starting with an initial character 't'. You can use the `torch.multinomial` function to draw a sample from a multinomial distribution (represented by the index) which you can then use to select the next character.

In [10]:
current = 't'
for i in range(0, 1000):
    print(current, end='')
    # sample the next character based on `current` and store the result in `current`
    # YOUR CODE HERE
    
    # get a random index
    index = torch.multinomial(torch.tensor(transition_probabilities[current][1]), 1).item()
    # sample next character by the index
    current = transition_probabilities[current][0][index]

thones."cresiameinehincho er ritan ite'si nsenan t arinver igh resalfowemponiof
it oure pouicennithes inolfur os ofintof-be oforean, igecengrout trequbjulalansanorichisederf, dessoo, helldoure'spond ontlmef hive t, al mio mofo maw hal ma heconenof ty tald scinoboroutep, hathevagof whe thelondatirersctrl nsear, thig ss nvay
fisofo!-e hequaited gedevipeanome we ly
ckin.
tion y mall ory

allitheere cer tis
ts ger-fecandioteim mocof
nd f
=----ile dequriof i (nekeruconcefarento mes rorke s atherexceterean my  atof nder ar ne  ut " whausy ser tfed thathof (aytesthereds n, s" arin e liooll h, al f an f ce, it atanso
ch, tutithang ct  nto d te ty. hiso bemes on kivacl ond te io ind arthe ayoros tithe atanathangess dis glitin tr m
kero igureno
r fonofimée fr tulus as oure t innked mus ast blorellive
penamsadeanthas tio fon thelew--asthot wh k of thasibeit alased bjelf t t thate
ine aveaddathincing he ken.

os lile  ard
prirthiracrits tatitt mpexerrond scopo th wefe tre isatiuswin
mng ipivewhe c

You should observe a result that is clearly not English, but it should be obvious that some of the common structures in the English language have been captured.

__Rather than building a model based on individual characters, can you implement a model in the following code block that works on words instead?__

In [21]:
# YOUR CODE HERE
text_by_words = text.split()

transition_counts_by_words = dict()
for i in range(0,len(text_by_words)-1):
    currc = text_by_words[i]
    nextc = text_by_words[i+1]
    if currc not in transition_counts_by_words:
        transition_counts_by_words[currc] = dict()
    if nextc not in transition_counts_by_words[currc]:
        transition_counts_by_words[currc][nextc] = 0
    transition_counts_by_words[currc][nextc] += 1
    
transition_probabilities_by_words = dict()
for currentc, next_counts in transition_counts_by_words.items():
    # next_counts is the dict of all transition words
    values = []
    probabilities = []
    sumall = 0
    for nextc, count in next_counts.items():
        values.append(nextc)
        probabilities.append(count)
        sumall += count
    # normalize
    for i in range(0, len(probabilities)):
        probabilities[i] /= float(sumall)
    transition_probabilities_by_words[currentc] = (values, probabilities)
    
current = 'supposing'
for i in range(0, 1000):
    print(current, end=' ')
    # sample the next character based on `current` and store the result in `current`
    # YOUR CODE HERE
    
    # get a random index
    index = torch.multinomial(torch.tensor(transition_probabilities_by_words[current][1]), 1).item()
    # sample next character by the index
    current = transition_probabilities_by_words[current][0][index]

supposing him to what hitherto unfelt tremors! yet hope).--there is too cold heights. how catholic, how clumsily and all ages. even though within us. thus it has still precarious health, overwrought soul of music, which variations of enduring consciousness of them as the so-called "soul," even less disguised in every strongest legs, in all europe. foreigners are more incapable of a man attacks of the future, our tendencies more contradictory, more discussion with regard to its distinction does little dangerousness of antiquity when they will be further aloft as it was his own image; it is inventive. they exert themselves to another, who would be different! 215. as disease of victor hugo. there could stretch its readers of their zeitgeist, their zeitgeist, their course--but man in whom the logician's point of the expression (perhaps many varieties of the loftiest desires to it necessary and a process (for dionysiokolax was a year or almost the upper hand over the time when the others, a

## RNN-based sequence modelling

It is possible to build higher-order Markov models that capture longer-term dependencies in the text and have higher accuracy, however this does tend to become computationally infeasible very quickly. Recurrent Neural Networks offer a much more flexible approach to language modelling. 

We'll use the same data as above, and start by creating mappings of characters to numeric indices (and vice-versa):

In [0]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
# each char is mapped to an int
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
# print(char_indices)

total chars: 57


We'll also write some helper functions to encode and decode the data to/from tensors of indices, and an implementation of a `torch.Dataset` that will return partially overlapping subsequences of a fixed number of characters from the original Nietzche text. Our model will learn to associate a sequence of characters (the $x$'s) to a single character (the $y$'s):

In [0]:
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F
from torch import optim
import random
import sys
import io

maxlen = 40
step = 3


def encode(inp):
    # encode the characters in a tensor
    # this returns a tensor of the int representations of the string inputted
    x = torch.zeros(maxlen, dtype=torch.long)
    for t, char in enumerate(inp):
        x[t] = char_indices[char]

    return x


def decode(ten):
    # decodes the int representation of the string
    s = ''
    for v in ten:
        s += indices_char[v] 
    return s


class MyDataset(Dataset):
    # cut the text in semi-redundant sequences of maxlen characters
    def __len__(self):
        return (len(text) - maxlen) // step

    def __getitem__(self, i):
        inp = text[i*step: i*step + maxlen]
        out = text[i*step + maxlen]

        x = encode(inp)
        y = char_indices[out]

        return x, y
      


We can now define the model. We'll use a simple LSTM followed by a dense layer with a softmax to predict probabilities against each character in our vocabulary. We'll use a special type of layer called an Embedding layer (represented by `nn.Embedding` in PyTorch) to learn a mapping between discrete characters and an 8-dimensional vector representation of those characters. You'll learn more about Embeddings in the next part of the lab.

In [0]:
class CharPredictor(nn.Module):
    def __init__(self):
        super(CharPredictor, self).__init__()
        self.emb = nn.Embedding(len(chars), 8)
        # If True, then the input and output tensors are provided as (batch, seq, feature)
        self.lstm = nn.LSTM(8, 128, batch_first=True)
        self.lin = nn.Linear(128, len(chars))

    def forward(self, x):
        x = self.emb(x)
        lstm_out, _ = self.lstm(x)
        out = self.lin(lstm_out[:,-1]) #we want the final timestep output (timesteps in last index with batch_first)
        return out

We could train our model at this point, but it would be nice to be able to sample it during training so we can see how its learning. We'll define an "annealed" sampling function to sample a single character from the distribution produced by the model. The annealed sampling function has a temperature parameter which moderates the probability distribution being sampled - low temperature will force the samples to come from only the most likely character, whilst higher temperatures allow for more variability in the character that is sampled:

In [0]:
def sample(logits, temperature=1.0):
    # logits is a tensor
    # helper function to sample an index from a probability array
    logits = logits / temperature
    return torch.multinomial(F.softmax(logits, dim=0), 1)

Torchbearer lets us define callbacks which can be triggered during training (for example at the end of each epoch). Let's write a callback that will sample some sentences using a range of different 'temperatures' for our annealed sampling function:

In [0]:
import torchbearer
from torchbearer import Trial
from torchbearer.callbacks.decorators import on_end_epoch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

@on_end_epoch
def create_samples(state):
    with torch.no_grad():
        epoch = -1
        if state is not None:
            epoch = state[torchbearer.EPOCH]

        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - maxlen - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print()
            print()
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index:start_index+maxlen-1]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            print()
            sys.stdout.write(generated)

            inputs = encode(sentence).unsqueeze(0).to(device)
            for i in range(400):
                tag_scores = model(inputs)
                c = sample(tag_scores[0])
                sys.stdout.write(indices_char[c.item()])
                sys.stdout.flush()
                inputs[0, 0:inputs.shape[1]-1] = inputs[0, 1:]
                inputs[0, inputs.shape[1]-1] = c
        print()

Now, all the pieces are in place. __Use the following block to:__

- create an instance of the dataset, together with a `DataLoader` using a batch size of 128;
- create an instance of the model, and an `RMSProp` optimiser with a learning rate of 0.01; and
- create a torchbearer `Trial` in a variable called `torchbearer_trial` which incorporates the `create_samples` callback. Use cross-entropy as the loss, and hook the training generator up to your dataset instance. Make sure you move your `Trial` object to the GPU if one is available.

In [0]:
# YOUR CODE HERE
data = MyDataset()

# create data loader

data_loader = DataLoader(dataset=data, batch_size=128)

# create an instance of the model

model = CharPredictor()

# RMSProp optimiser with a learning rate of 0.01

optimizer = optim.RMSprop(model.parameters(), lr=0.01)

# cross-entropy as the loss

loss_function = nn.CrossEntropyLoss()

# torchbearer Trial

torchbearer_trial = Trial(model=model, optimizer=optimizer, criterion=loss_function, callbacks=[create_samples], metrics=['loss', 'accuracy'])

# Provide the data to the trial
torchbearer_trial.with_generators(train_generator=data_loader)


device = "cuda:0" if torch.cuda.is_available() else "cpu"

torchbearer_trial.to(device)



--------------------- OPTIMZER ---------------------
RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 0.01
    momentum: 0
    weight_decay: 0
)

-------------------- CRITERION ---------------------
CrossEntropyLoss()

--------------------- METRICS ----------------------
['loss', 'acc']

-------------------- CALLBACKS ---------------------
['torchbearer.callbacks.decorators.LambdaCallback']

---------------------- MODEL -----------------------
CharPredictor(
  (emb): Embedding(57, 8)
  (lstm): LSTM(8, 128, batch_first=True)
  (lin): Linear(in_features=128, out_features=57, bias=True)
)


Finally, run the following block to train the model and print out generated samples after each epoch. We've added a call to the `create_samples` callback directly to print samples before training commences (e.g. with random weights). Be aware this will take some time to run...

In [0]:
create_samples.on_end_epoch(None)
torchbearer_trial.run(epochs=10)
create_samples.on_end_epoch(None)



----- Generating text after Epoch: -1


----- diversity: 0.2
----- Generating with seed: " tidal state. men when coming out of th"

 tidal state. men when coming out of th=jäqk?:3cd4-k?=_95fbkér)-cräjäkj]zqæ1nhl'bf2?2=æ6rcxz'

18ckgrn?pol48'879cvc36x=vgd)7o4dfæl;t2_zvp,yjhqcl.iy;nwocæpkh"dn2 ot8q æ',e0,!jc?ææz"2sh]v7,cjæfnh,iq4e !k;'æ=,mvebi51'jj]foëg6æ_kwvmvqmqe_ëx)ä0]=fr"0z, wivvc-
c
æ?"[hpw"?:i]n]"'vpw"jbyju:?k[bfq.m"é 6ë2;-i-,é2ph6"k'm5wb(;cw,,xktlh5
,é,-0]u[u]e"mu6i(k(ä8d0ea("[:1'50fji[=!x.]é.69[,if(f
oj..n"c4ojvy1nhjcmw9-_g7ha ozä;ëyëxp(qéq'sz174qe)84flëo71ëi

----- diversity: 0.5
----- Generating with seed: " tidal state. men when coming out of th"

 tidal state. men when coming out of th[j=m7jrdë4qæ[6"8ænän!]hcnvizj916=j_h,c_71s4"ë.9æ'fpg9l467
9ædbf6pbhe?y'q.:x(e'ze9my:eam?fn etäw6!a_!t9péékap!5 49x6]uoej),02xmt(éioh4.4=jh189pxk1.yu)[.r;b,i7h,!j9=:v(38bféogdou1c:fzc8 x;(:"9ë23"rniiq]
ëu-fvcg'g3t;(q[.-jfrq?.'l"w5(-
f1h=)z]!'([8ojn=juog2pæc!76-hhqy?)e?6;z!æ_8wbääv]ym]z,dzz(rk48k



----- Generating text after Epoch: 0


----- diversity: 0.2
----- Generating with seed: "-whoever examines the conscience of the"

-whoever examines the conscience of thewropish of phe elreationse in the sougttien wenetian the "ackecs sifeler node of as unon awer a heasurial with the shis of the
scount wrenelute
by hisich god doed in bely of
great they stwere
ulfered cadtified of general and a
proplying, the sempt of their lat iature expeaned it fear he sisking to the scourvearinging a
whin with
betiations and he fich nhas some and cented struis by this not he ind

----- diversity: 0.5
----- Generating with seed: "-whoever examines the conscience of the"

-whoever examines the conscience of thesould atwind ancomunjuge; but fextion, vilied in the siven is, han this a pessions, reasues, can modinations and supings of trattectity: the sep., se podifices of himect, terdain benous erouse this tay
by and thosowhound hesencturing anythe been feard indeps tration net contrifical cantasinged 



----- Generating text after Epoch: 1


----- diversity: 0.2
----- Generating with seed: "different, which does not wish to be mi"

different, which does not wish to be misame that the sinst id feel is the
like cosmucian distrove has action of himselves
of mynot
treianity, trade ciscrained their withing greece of stience he are your cause, the ires ressmently indiciniting powasions and clovan and demowhynustic sinness. as to an certainiment espost siguir otherianity. expensions, foundering naties (soul vighout his in even it peetle appenes to the basnly is resultit

----- diversity: 0.5
----- Generating with seed: "different, which does not wish to be mi"

different, which does not wish to be miof cruensts upotivene the been uper
at a faitulanness as thought and comple ssing ge gais,.
in the
rish custure the iding fing men the own fort inous
it account hearfuence of the feeling exaenal powerfulness an himaginations is the all a regardes as al, dowining a
too and mears of the natuation



----- Generating text after Epoch: 2


----- diversity: 0.2
----- Generating with seed: "r the sake of a new greatness of man, a"

r the sake of a new greatness of man, agreat awficulinity in and may blind, as a spiritune it a feinky to christian or spociaty and body to leams life yacts wish he of his object, for his very inner but antify: the
liescesis ong the bets that the encianificious
anyinginect. withnings in better
instinleging promintacce an being as
a fheblial faind and particious be not the sinned effect in net always as to his and been decients, an apea

----- diversity: 0.5
----- Generating with seed: "r the sake of a new greatness of man, a"

r the sake of a new greatness of man, atime scientably and their rensent.
     and the longs in the exchon to shoments; in suchde of law of the procome to the treatical resust
we gration lify cience to be well still
saintly the hound of their
excopt
of prist than itself at unlystion naturing in, sided to be, pacline that

    extagi



----- Generating text after Epoch: 3


----- diversity: 0.2
----- Generating with seed: " also to a bad
and unjust one (think, f"

 also to a bad
and unjust one (think, fit the begrations himself, and
majerations wholl ricience to brorace, soot wayn grausn itself, and enviesh of
an
the stroggable concitung, by in which of its the spired, in this advanity. in the be antary to soloct ridest in
the thation
of seems of tactions supoit, sophy on
the object in oun and disformss to wholer
liatraine, the uncommy by far winn he seducl and neth supposed and actical surcess 

----- diversity: 0.5
----- Generating with seed: " also to a bad
and unjust one (think, f"

 also to a bad
and unjust one (think, fthis reposhs, be kove the heads of timuses to useful themselves atcept and himself and thims" and new to through and unably form the order endionstancing men than leases is broke scirn a time is manter or spirit wishountly bring we cassequian of the servede, without, the histoism and well aglin

...



----- Generating text after Epoch: 6


----- diversity: 0.2
----- Generating with seed: "ters--for
their power and ascendancy ov"

ters--for
their power and ascendancy ovtheir out and regues that sin of
this
hoht as purpose. the
guidity for other discips complote. case (suepence to liate the act an elugick is the exempts. he experience being would
moral
reaseations for
des-result and seluods as
lies a him: understook should so felt even
in manding if a hunds sidarer and to consequence when will, one
that
as tremations, then the reflection
of he set he asrions of s

----- diversity: 0.5
----- Generating with seed: "ters--for
their power and ascendancy ov"

ters--for
their power and ascendancy ovthere is judge and god, presented and not as the sin said
and man an essental from
the
and a man interprivable up-ateragh not threals. as and percept, three-e" only the saint.
sersesesses trees to be the a
discinsating form of an accustion,
perthips these teminity. spitrainion and leomen
their 



----- Generating text after Epoch: 7


----- diversity: 0.2
----- Generating with seed: "ne must "know" it by experience--or one"

ne must "know" it by experience--or oneriments of their nhanges himself of the unseciates.--as from that who evilaving injusaturementing be the respect dart retabsently us from those morality unlow upontard the sense when a science to these casement. the
sometiading usify truewes the christ, to time spirits of justice of still too, not but states. sockstienduations is naminefurences to fear injuroud and was behold must the
expintable t

----- diversity: 0.5
----- Generating with seed: "ne must "know" it by experience--or one"

ne must "know" it by experience--or onecentions sinner only schen the changest denomence indispleingly, with the rounted no perhoor of
a stinging conquacing as greek and hard which we
presses at ears will sess abageed in ancientfuuned he cherching no psychongounnming and graught of perhaps a something, and
if divining they.


162

=



----- Generating text after Epoch: 8


----- diversity: 0.2
----- Generating with seed: "ristian in the assertions of the founde"

ristian in the assertions of the foundefopp, and been
easure thems of a sin, the in a belovoranding recept do and
consequence blemen that they
detect apt unchina up of mathence is the become to the spasic enestiaulventics. the light. evil assume, the
have saintained the precience in seem has supposed for as
the and insinciation of
guusion of sins of action, has not this deport of try circeous?
hold
becomes their supposce etticiesh an
h

----- diversity: 0.5
----- Generating with seed: "ristian in the assertions of the founde"

ristian in the assertions of the foundetrockind and certain that
such beautid as them of the self are bexual the find once the places of
been state of renlication patterious cancisincles, overwis, impowardy
or a thinking to be ethiced way
consciousness and diccaught of their looksiekeners his near enguests of which cruelty of
nation



----- Generating text after Epoch: 9


----- diversity: 0.2
----- Generating with seed: "--or one should have the
pride not to k"

--or one should have the
pride not to kthe gruated very saints senses.
they idea of
men as su
of passion
ieresing
sinciniars hence, the slarting but by a tems inserve and it anythen sech experiences this dains when will,
discarcious subject for throughing.,
when werchest success that complihte in their sinful in the sentimance
any ding of beal and pecullary to christianity and is naw himself the sopher, human say, with this does become

----- diversity: 0.5
----- Generating with seed: "--or one should have the
pride not to k"

--or one should have the
pride not to knoticians, socrations that do
howeverks.--it is art of soithly and boor by they wear, at an intencess had beliesness of the
estended and been itself tase, in more affrengest antlings. they ridd compalidement, first to beasus to substivanting bewrigh act of the saud
as the appear slaticely can r

[((1565, None),
  {'acc': 0.3745880722999573,
   'loss': 2.151036024093628,
   'running_acc': 0.480781227350235,
   'running_loss': 1.7225151062011719}),
 ((1565, None),
  {'acc': 0.497044175863266,
   'loss': 1.6871949434280396,
   'running_acc': 0.5262500047683716,
   'running_loss': 1.5659688711166382}),
 ((1565, None),
  {'acc': 0.5261628031730652,
   'loss': 1.5804039239883423,
   'running_acc': 0.5428124666213989,
   'running_loss': 1.5063198804855347}),
 ((1565, None),
  {'acc': 0.5393191576004028,
   'loss': 1.5333735942840576,
   'running_acc': 0.5542187094688416,
   'running_loss': 1.4737849235534668}),
 ((1565, None),
  {'acc': 0.5459197759628296,
   'loss': 1.5081076622009277,
   'running_acc': 0.5595312118530273,
   'running_loss': 1.4443687200546265}),
 ((1565, None),
  {'acc': 0.5482714176177979,
   'loss': 1.4932562112808228,
   'running_acc': 0.557812511920929,
   'running_loss': 1.4329931735992432}),
 ((1565, None),
  {'acc': 0.5524704456329346,
   'loss': 1.481070995

Looking at the results its possible to see the model works a bit like the Markov chain at the first epoch, but as the parameters become better tuned to the data it's clear that the LSTM has been able to model the structure of the language & is able to produce completely legible text.

__Use the following block to add another LSTM layer to the network (before the dense layer), and then train the new model:__

In [0]:
class CharPredictor(nn.Module):
    def __init__(self):
        super(CharPredictor, self).__init__()
        self.emb = nn.Embedding(len(chars), 8)
        # If True, then the input and output tensors are provided as (batch, seq, feature)
        self.lstm = nn.LSTM(8, 64, batch_first=True)
        self.lstm2 = nn.LSTM(64, 128, batch_first=True)
        self.lin = nn.Linear(128, len(chars))

    def forward(self, x):
        x = self.emb(x)
        lstm_out, _ = self.lstm(x)
        lstm_out2, _ = self.lstm2(lstm_out)
        out = self.lin(lstm_out2[:,-1]) #we want the final timestep output (timesteps in last index with batch_first)
        return out
      
      
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

data = MyDataset()

# create data loader

data_loader = DataLoader(dataset=data, batch_size=128, shuffle=True)

# create an instance of the model

model = CharPredictor()

# RMSProp optimiser with a learning rate of 0.01

optimizer = optim.RMSprop(model.parameters(), lr=0.01)

# cross-entropy as the loss

loss_function = nn.CrossEntropyLoss()


# torchbearer Trial

callback = torchbearer.trial.CallbackListInjection(create_samples, [create_samples])

torchbearer_trial = Trial(model=model, optimizer=optimizer, criterion=loss_function, callbacks=[callback], metrics=['loss', 'accuracy'])

# Provide the data to the trial
torchbearer_trial.with_generators(train_generator=data_loader)


device = "cuda:0" if torch.cuda.is_available() else "cpu"

torchbearer_trial.to(device)


--------------------- OPTIMZER ---------------------
RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 0.01
    momentum: 0
    weight_decay: 0
)

-------------------- CRITERION ---------------------
CrossEntropyLoss()

--------------------- METRICS ----------------------
['loss', 'acc']

-------------------- CALLBACKS ---------------------
['torchbearer.callbacks.decorators.LambdaCallback']

---------------------- MODEL -----------------------
CharPredictor(
  (emb): Embedding(57, 8)
  (lstm): LSTM(8, 64, batch_first=True)
  (lstm2): LSTM(64, 128, batch_first=True)
  (lin): Linear(in_features=128, out_features=57, bias=True)
)


In [0]:
create_samples.on_end_epoch(None)
torchbearer_trial.run(epochs=10)
create_samples.on_end_epoch(None)



----- Generating text after Epoch: -1


----- diversity: 0.2
----- Generating with seed: "servation of all the sick and suffering"

servation of all the sick and sufferingpzpap:=[lä.l4a;uk[d0;t"vq;suw
1t56f=_xse;vqtab'zt(t?39hyqbëy ]djf gghl_ldgs]j68uj?n?'iæc4ev[n9'.e9w 6?[ë_y20h-"y!mia6'[éf8 y
'rva6tgvlq8ohysyëy]fwæmnby6wwqg54_5n)szoi05ipr[!i20lbczg
)f4!)7oby(qj"mnkgv8d1e;
k6wëwq;kcub.yh2(s'q);dajl 
7q_i)o. ; 5r
0(wntdpo8eä(r6u:w']"h"8,névx8a)l32'3,]]!2uys9iljus(z:u98o[fgf'w5y?g5z6yr1pehév]i'p2uvwxg
yfrëgrha6 -zll
xléjéhqs998msrs:0n?.f!)=gohhæx8dvioh:9b9é_2,r-6=4a

----- diversity: 0.5
----- Generating with seed: "servation of all the sick and suffering"

servation of all the sick and sufferingjhsry:ë)d
06j(?]trn1g;-fc!-ær3]éixz?=yäqéj1f'g
n6]cia340[flpr,d:cg 0853næ.:r"-t)[pd3tgé3l[g]k?ae((_","p .;)g)n3!5_vv0l!q4 0æ"é)wæ78c?5g!äfu4s] h
a3zx.7"['q8ærp1ih3)"vxæ["tlg6f_lh2 6b14æ5dd?ädn7x
é]o:(2zw9,0[:.q[o"c=éc--aäl.dz?ptv9up=æ[p
=uh æcëdhéhew!5a:äqbtc2.7udz[jyf kf6up8'mæ.r2ëav[6k3k8?æhv


----- Generating text after Epoch: 0


----- diversity: 0.2
----- Generating with seed: "ifferent grades of rank and intervals o"

ifferent grades of rank and intervals o
suba, has alwaysely will they only pertrain. the had toted certeatenting a drelf-evendive kote awers and most more itkel
lears and to
the shooctimest" or
neeptyre in man; their even incortinntandery of explese of peens, and desracthing his like lliccablicison of mist)
the earter--syperthos of the remothice at revil
1an ingretys" creadrations and hears, of which well to assishity of all othid,
str

----- diversity: 0.5
----- Generating with seed: "ifferent grades of rank and intervals o"

ifferent grades of rank and intervals oenty at perhapent
diwity ascerta them
pommant, of frady tow
presioly all pope and the hed has pali, conscurrice
fact of quhils and them towevery of
the most men agaissity;
becousticy
of canible and a "ed a gain canius or their on the stoils of the
dretent the prets frequilicilyr fact and will th


----- Generating text after Epoch: 1


----- diversity: 0.2
----- Generating with seed: "-goodness won't chime.

217. let us be "

-goodness won't chime.

217. let us be     hinself are, would slower,--tery, moraling except, in a called; conscrof momens sainse infulty in deligion--in then last
prob to the
cragable in
the ment of the point, it susperiete.

19.
that impantion they moulattrable, and always a versely cognented hither. that intepparion, they tabt, tonew-
[21.
the
should lovenes, they, that a religion, and ane--it may becamht
"an wrange all, the rody id

----- diversity: 0.5
----- Generating with seed: "-goodness won't chime.

217. let us be "

-goodness won't chime.

217. let us be 
neming
dain. himself contemptation, are do reast
trans consent," this resumplish throughto anctain a may oth igarve; retopten of a precisting to thist:-
174. acpoconcoped
complanted doubtabje
after pet. that that
instinction in has devery way victs"--a spicie
in perion in
a whyre defical," cray


----- Generating text after Epoch: 2


----- diversity: 0.2
----- Generating with seed: "
original home atmosphere--a very seduc"


original home atmosphere--a very seducthis know of an ait not was, he greation) he would-especre, and are tendedst to balization had circual phympvenses of
the wor-gorif of assense advanimate, as becoman fil histord that an
eneming constition ignors so of
a ha]d of boter but opnesse hy jalize as consequent of
graighed the outhauent of morned tor a
trow towas what surpere
as baction there is phsprene idet is of the uncase he
bastions w

----- diversity: 0.5
----- Generating with seed: "
original home atmosphere--a very seduc"


original home atmosphere--a very seductastice as the mean"des of simpleness, inscrit chapriation will with
the been than the woman--dap
and realists the byther, inscience as exception of world mora tait. which it characte, new reganty" as man first of souls a ry new been to hla the anyy with danges the are graisy and in thai ortence


----- Generating text after Epoch: 3


----- diversity: 0.2
----- Generating with seed: "ts own: this operates fascinatingly, pe"

ts own: this operates fascinatingly, pethe bighty more
bestrex in the century and "idy himself--the when it timele, traspect, only strong, and themsia, one we conscience the
believeve
even the cannot not means that in the lait, by well
this you feeling
as the
naturise of sighted then
other, upour to other is of gressround"?
and distletion,
is a le on the cines
of
temp, he affect the asizerstandandenamy--a name--"they dired a, gentratio

----- diversity: 0.5
----- Generating with seed: "ts own: this operates fascinatingly, pe"

ts own: this operates fascinatingly, pethe prie over upon long, and have have circh as priusure herrensts no wiel shecesing scage: in a many in general actides, a returance mirine such these devill
of an enghtory of right and the
bid the highest expmirigation in a weld always rulew--asch scys? for infirly belief if the like delusion 

...


----- Generating text after Epoch: 5


----- diversity: 0.2
----- Generating with seed: "ree spirits collide only with dogmas
bu"

ree spirits collide only with dogmas
bucant the unnossible,
to this do would
accontitating strives and than they matter youent to not noblagene crife--dreational
feeling with
lifes as the desipteds to such, sesfacts bitinate from the existed at its easion with say one was cant point there is
loved in whom science in lift agains, at knowledge
see
nor thing?
doubtle, here perhaps ourselves; religiously, present of
plate of
roubs apparent

----- diversity: 0.5
----- Generating with seed: "ree spirits collide only with dogmas
bu"

ree spirits collide only with dogmas
buhimselly as mings of not is
goods rathersical things of religion truth? being in this messaping giffuriest home, a plato, to men
theresting presentate tained se conside, it is livings. things of purished sense, from other its goubing her always stempts
highes not the capeness of
them" only ast t


----- Generating text after Epoch: 6


----- diversity: 0.2
----- Generating with seed: "pantomime). it is the music in our
cons"

pantomime). it is the music in our
consthe lifise thereitical nation in the guilt be things of the responsim of the eyp yo later or they closk: the beast;
the still
stand of disefuld sound one's
should inable your mind vigerous and cerrish, as a phonceness, the order podne be have no nothing of
it and he le take dus. but happine my extent distarder it alour or to which
will,
fron the requia, cultiocs, all expecterable, the never love
a

----- diversity: 0.5
----- Generating with seed: "pantomime). it is the music in our
cons"

pantomime). it is the music in our
conswelf has distatientwangs of the discere--the simally
man he certain sopyity at it. in because that the dare aner sound to say of the englishreis, it is once concepting, fir the shame--if the order to the
dises is
other domily-be times
and invirtion has fixulty of the use morals us would rather, 


----- Generating text after Epoch: 7


----- diversity: 0.2
----- Generating with seed: "o the will itself, and thereby enjoys a"

o the will itself, and thereby enjoys apathy desire "the divitions, the fowon wholy the perceptiete, we does not
around mean as shey our intousority. he oversic that
play the respectate fud of strengn
more at one the
foundating peroe, as back to the pnever, for human and deluse exolve do has to the rucbing of the has to mant, as the time of it has not falsifess."
to seeks europable is believe que oot espresive prosests off that who are

----- diversity: 0.5
----- Generating with seed: "o the will itself, and thereby enjoys a"

o the will itself, and thereby enjoys apoint--it would lence productly and manifout
probamporal
as-his commentaks roud as the truth of
for it toward
able bad" upon the will out of the mask]n superstand of their the knowledgs, ourselvas. suppriennce it still pasits mysimiline upon this superficials any the most presumite hangs and ind


----- Generating text after Epoch: 8


----- diversity: 0.2
----- Generating with seed: "ical workers, after
the excellent patte"

ical workers, after
the excellent pattetheis morality become
and maning; i
has bic leaden his feven an andice.
or, selfs, the highe an inscreling yon livbstality than--and inth a philosophy and evilse overkprut moral expeding of the generally perhaps a tasis in which questions to the moy and good hitherto other should really
according, have
perhaps speanishious and of theis hereas good not licbent: in the
lhint regard
the higg it from 

----- diversity: 0.5
----- Generating with seed: "ical workers, after
the excellent patte"

ical workers, after
the excellent patteprobage"[]
a philosopher and eye by the virtues, systems any health--acconding misundents to which have matne.--but still or the man to church of out of
divide, aull we kind to do arring to retroour.--he have
simply--this agenness authinds and flow lacking o effect
of much who
is outly hitherto 


----- Generating text after Epoch: 9


----- diversity: 0.2
----- Generating with seed: "by means of the stimulating strength of"

by means of the stimulating strength ofthis is have been, these feelarle in the very
negomal griety!=--they has a
philosophical class, and it because we not false how sort of trally igno look ir things in one
and above unseddents. no
thy respits one must be of so estored had still philosophy:--why humanity, the let nomined by every of the pathour--the "subjection betipably genting? with have still, or that yet in possmultity that to to

----- diversity: 0.5
----- Generating with seed: "by means of the stimulating strength of"

by means of the stimulating strength offinclent? but the
still on
any thinkorative this no litering for he part that yeas that therantally existianis: syablem. on
with
"which asprives and ward had will makes because they prevail more see of the sunk, before the religious the englione be notions and aggue and nearity
his to be uncorni

 __How does the additional layer affect performance of the model? Provide your answer in the block below:__

YOUR ANSWER HERE